In [ ]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import os, yaml
import numpy as np

In [ ]:
root = os.path.abspath(os.path.join(os.getcwd(),'..'))

In [ ]:
sites = ['extras','ganga_damodar','godavari','kali','kaveri','krishna','mahanadi','narmada','penner','sharavati','tapi']

In [ ]:
sites_data = yaml.load(open(os.path.join(root,'bin','experiments-lowest.yaml'),'r'),Loader=yaml.SafeLoader)

In [ ]:
ds_gconv = {}
for site in sites:
    #print (sites_data[site]['gconv'].split('-'))
    el = os.path.splitext(os.path.split(sites_data[site]['gconv'])[-1])[0][-2:]
    #el = str(int(el)+10)
    ds_gconv[site] = xr.load_dataset(os.path.join(root,'data','final_preds',f'{site}-gconv-{el}-preds.nc'))
ds_noconv = {}
for site in sites:
    el = os.path.splitext(os.path.split(sites_data[site]['no_gconv'])[-1])[0][-2:]
    ds_noconv[site] = xr.load_dataset(os.path.join(root,'data','final_preds',f'{site}-no_gconv-{el}-preds.nc'))

In [ ]:
all_gconv = xr.merge([ds_gconv[site] for site in sites])

In [ ]:
all_gconv

In [ ]:
slice_edges = [-1.,-0.2,-0.15,-0.1,-0.05,0,0.05,0.1,0.15,0.2,1]
obs_slices = [slice(s0,s1,None) for s0,s1 in zip(slice_edges[:-1],slice_edges[1:])]

In [ ]:
list(np.arange(0,1.1,0.1))

In [ ]:
slice_edges = [np.percentile(all_gconv['obs'].values.flatten()[~np.isnan(all_gconv['obs'].values.flatten())],_p*100) for _p in np.arange(0,1.1,0.1)]
obs_slices = [slice(s0,s1,None) for s0,s1 in zip(slice_edges[:-1],slice_edges[1:])]

In [ ]:
obs_slices

In [ ]:
-7e-3

In [ ]:
#slice_edges = [-1.,-0.007,-0.005,-0.003,-0.0015,-0.00075,0,0.006,1]
slice_edges = [-1.,-0.005,-0.0015,0,1]
obs_slices = [slice(s0,s1,None) for s0,s1 in zip(slice_edges[:-1],slice_edges[1:])]

In [ ]:
for s in obs_slices:
    print(((all_gconv['obs']>s.start)&(all_gconv['obs']<=s.stop)).sum())

In [ ]:
all_gconv['ci'] = all_gconv['ci-95+'] - all_gconv['ci-95-']

In [ ]:
ci_obs_data = []
for s in obs_slices:
    mask = ((all_gconv['obs']>s.start)&(all_gconv['obs']<=s.stop)).values
    all_gconv.coords['mask']= (('date','step','site'),mask)
    print (s)
    #print(all_gconv.where(all_gconv.mask>0)['ci'].mean(dim=['date','step']))
    d = all_gconv.where(all_gconv.mask>0)['ci'].mean(dim=['date','step']).values
    ci_obs_data.append(
        d[~np.isnan(d)]
    )

In [ ]:
len(ci_obs_data)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,3))
collections = ax.violinplot(ci_obs_data, showmeans=True)

In [ ]:
step_edges = [0,5,15,30,50,75,90]
step_slices = [slice(s0,s1,None) for s0,s1 in zip(step_edges[0:-1],step_edges[1:])]

In [ ]:
s = step_slices[0]

In [ ]:
ci_step_data = []
for s in step_slices:
    ci_step_data.append(
        all_gconv.sel({'step':s})['ci'].mean(dim=['step','date']).values
    )

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,3))
collections = ax.violinplot(ci_step_data, showmeans=True)

In [ ]:
# correction factor - how much stretch to hit obs
all_gconv

In [ ]:
obs - sim_mean / ci-95+ - sim-mean

In [ ]:
all_gconv['alpha+'] = (all_gconv['obs'] - all_gconv['sim-mean']) / (all_gconv['ci-95+'] - all_gconv['sim-mean'])

In [ ]:
all_gconv['alpha-'] = (all_gconv['obs'] - all_gconv['sim-mean']) / (all_gconv['ci-95-'] - all_gconv['sim-mean'])

In [ ]:
all_gconv.coords['maskpos'] = (('date','step','site'),(all_gconv['obs']>all_gconv['sim-mean']).values)
all_gconv.coords['maskneg'] = (('date','step','site'),(all_gconv['obs']<all_gconv['sim-mean']).values)

In [ ]:
all_gconv['alpha+'].shape

In [ ]:
all_gconv['alpha_combined'] = (('date','step','site'),np.zeros(all_gconv['alpha+'].shape))

In [ ]:
all_gconv['alpha_combined'] += all_gconv.where(all_gconv.maskneg>0)['alpha-'].fillna(0)

In [ ]:
all_gconv['alpha_combined'] += all_gconv.where(all_gconv.maskpos>0)['alpha+'].fillna(0)

In [ ]:
all_gconv.coords['mask_combined'] = all_gconv['alpha_combined']>0

In [ ]:
s

In [ ]:
np.isfinite

In [ ]:
alpha_step_data = []
for s in step_slices:
    d = all_gconv.where(all_gconv.mask_combined>0)['alpha_combined'].sel({'step':s}).mean(dim=['step','date']).values
    alpha_step_data.append(
        d[np.isfinite(d)]
    )

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,3))
collections = ax.violinplot(alpha_step_data, showmeans=True)
ax.set_yscale('log')

In [ ]:
#slice_edges = [-1.,-0.007,-0.005,-0.003,-0.0015,-0.00075,0,0.006,1]
slice_edges = [-1.-0.005,-0.0015,0,1]
obs_slice_labels = ['-1 : -5e-3','-5e-3 : -1.5e-3','-1.5e-3 : 0.','0. : 1.'] 

In [ ]:
c_face = '#cfe2f3ff'
c_edge = '#3d85c6'

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(12,6))
axs=axs.flatten()
collections0 = axs[1].violinplot(ci_obs_data, showmeans=True) # CI vs delta
collections1 = axs[0].violinplot(ci_step_data, showmeans=True) # CI vs step
collections2 = axs[2].violinplot(alpha_step_data, showmeans=True) # alpha vs step

for collection in [collections0, collections1, collections2]:
    vp = collection['bodies']
    for b in vp:
        b.set_facecolor(c_face)
    for e in ['cmaxes','cmins','cbars']:
        vp = collection[e]
        vp.set_edgecolor(c_edge)

axs[0].text(0.01,0.9,'(a)',fontsize=12, weight='bold',transform=axs[0].transAxes)
axs[2].text(0.01,0.9,'(b)',fontsize=12, weight='bold',transform=axs[2].transAxes)
axs[1].text(0.01,0.9,'(c)',fontsize=12, weight='bold',transform=axs[1].transAxes)
        
axs[1].set_xticks(list(range(1,5)))
axs[1].set_xticklabels(obs_slice_labels)
axs[0].set_xticklabels(['','0-5 days']+[f'{s.start}-{s.stop}' for s in step_slices[1:]])
axs[2].set_xticklabels(['','0-5 days']+[f'{s.start}-{s.stop}' for s in step_slices[1:]])
axs[2].set_yscale('log')
axs[3].axis('off')
axs[0].set_ylabel('Confidence Interval')
axs[2].set_ylabel('Correction Factor')
axs[2].set_xlabel('Prediction Horizon')
axs[1].set_xlabel('Daily Reserv. Vol. Change ($\sigma$)')
fig.savefig('./uncertainty_plots.pdf',bbox_inches='tight')

In [ ]:
all_gconv.where(all_gconv.mask_combined>0)['alpha_combined'].sel({'step':s}).mean(dim=['step','date']).values


In [ ]:
all_gconv.where(all_gconv.maskneg>0)

In [ ]:
# alpha+ where obs>mean & alpha+>1
# alpha- where obs<mean

In [ ]:
mask = ((all_gconv['obs']>s.start)&(all_gconv['obs']<=s.stop)).values

In [ ]:
all_gconv.coords['mask']= (('date','step','site'),mask)

In [ ]:
all_gconv.where(all_gconv.mask>0)['ci'].mean(dim=['date','step']).values

In [ ]:
ds_gconv['penner']